### Loading data (2023 year) 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
df = pd.read_csv('../complaints.csv')
df['Date received'] = pd.to_datetime(df['Date received'])

cols_to_consider = ['Product','Sub-product','Issue','Sub-issue','Consumer complaint narrative','Company public response','Company',
                    'State', 'ZIP code', 'Date received']
df_new = df[cols_to_consider]

df_new = df_new.dropna()

/var/folders/hq/f_rzh7j51m1ddfp3xw6s1qsr0000gn/T/ipykernel_73491/455528165.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../complaints.csv')


In [4]:
df_2023 = df_new[df_new['Date received'].dt.year.isin([2023])].reset_index(drop=True)

product_map = {'Credit reporting or other personal consumer reports' : 'Credit Reporting',
               'Credit reporting, credit repair services, or other personal consumer reports' : 'Credit Reporting',
               'Payday loan, title loan, personal loan, or advance loan' : 'Loans / Mortgage',
               'Payday loan, title loan, or personal loan' : 'Loans / Mortgage',
               'Student loan' : 'Loans / Mortgage',
               'Vehicle loan or lease' : 'Loans / Mortgage',
               'Debt collection' : 'Debt collection',
               'Credit card or prepaid card' : 'Credit/Prepaid Card',
               'Credit card' : 'Credit/Prepaid Card',
               'Prepaid card' : 'Credit/Prepaid Card',
               'Mortgage' : 'Loans / Mortgage',
               'Checking or savings account' : 'Checking or savings account'  
              }

df_2023.loc[:,'Product'] = df_2023['Product'].map(product_map)

In [5]:
df_2023.head()

,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Date received
0,Checking or savings account,Other banking product or service,Opening an account,Account opened without my consent or knowledge,Date : XXXX XXXXo : XXXX XXXX XXXX / XXXX XXXX...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,NC,27513,2023-12-29
1,Credit Reporting,Credit reporting,Problem with a company's investigation into an...,Investigation took more than 30 days,I have previously disputed this item with you ...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,MN,55124,2023-12-29
2,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,I kindly request that you update my credit rep...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,IL,60621,2023-12-28
3,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,I implore you to conduct a comprehensive inves...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NJ,08723,2023-12-28
4,Credit Reporting,Credit reporting,Incorrect information on your report,Information belongs to someone else,In accordance with the Fair Credit Reporting A...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",TX,77377,2023-11-27


In [6]:
df_2023.columns

Index(['Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Date received'],
      dtype='object')

In [9]:
df_2023.shape

(265315, 11)

### Complaint pre-processing

In [10]:
df_2023['complaint length'] = df_2023['Consumer complaint narrative'].apply(lambda x : len(x))

In [11]:
df_2023 = df_2023[df_2023['complaint length'] > 20]

complaints_to_exclude = ['See document attached', 'See the attached documents.', 'Incorrect information on my credit report', 'incorrect information on my credit report',
'please see attached file','Please see documents Attached','Incorrect information on my credit report.', 'Please see attached file', 'see attached',
'See attached', 'SEE ATTACHED DOCUMENTS', 'See Attached', 'SEE ATTACHMENT', 'SEE ATTACHMENTS', 
'XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX']

df_2023 = df_2023[~df_2023['Consumer complaint narrative'].isin(complaints_to_exclude)]

### Categories consideration

In [12]:
df_2023_subset = df_2023[['Consumer complaint narrative','Product','Sub-product','Issue','Sub-issue']]
df_2023_subset.shape

(264968, 5)

In [13]:
df_2023_subset['Product'].value_counts()

Product
Credit Reporting               213403
Credit/Prepaid Card             16319
Checking or savings account     15143
Debt collection                 11767
Loans / Mortgage                 8336
Name: count, dtype: int64

In [14]:
df_2023_subset['Sub-issue'].value_counts()

Sub-issue
Information belongs to someone else                                      57877
Reporting company used your report improperly                            48781
Their investigation did not fix an error on your report                  45407
Credit inquiries on your report that you don't recognize                 13150
Account status incorrect                                                 10271
                                                                         ...  
Issues with financial aid services                                           1
Threatened to turn you in to immigration or deport you                       1
Did not receive car title                                                    1
Problem adding money                                                         1
Cannot graduate, receive diploma, or get transcript due to money owed        1
Name: count, Length: 206, dtype: int64

In [15]:
sub_issues_to_consider = df_2023_subset['Sub-issue'].value_counts()[df_2023_subset['Sub-issue'].value_counts() > 500].index

In [17]:
len(sub_issues_to_consider)

44

In [18]:
reduced_subissues = df_2023_subset[df_2023_subset['Sub-issue'].isin(sub_issues_to_consider)]

In [19]:
reduced_subissues.shape

(248065, 5)

In [20]:
reduced_subissues['Sub-issue'].value_counts()

Sub-issue
Information belongs to someone else                                                      57877
Reporting company used your report improperly                                            48781
Their investigation did not fix an error on your report                                  45407
Credit inquiries on your report that you don't recognize                                 13150
Account status incorrect                                                                 10271
Account information incorrect                                                             9307
Was not notified of investigation status or results                                       9201
Investigation took more than 30 days                                                      8937
Personal information incorrect                                                            5900
Debt is not yours                                                                         2821
Deposits and withdrawals                

In [21]:
df_2023['Sub-product'].value_counts()

Sub-product
Credit reporting                              212160
General-purpose credit card or charge card     13689
Checking account                               12710
Other debt                                      3482
Loan                                            3239
I do not know                                   2832
Credit card debt                                2180
Federal student loan servicing                  2041
Store credit card                               1924
Conventional home mortgage                      1260
Other personal consumer report                  1243
Medical debt                                    1230
Savings account                                 1214
Other banking product or service                 952
Auto debt                                        823
FHA mortgage                                     532
Telecommunications debt                          482
Lease                                            438
Government benefit card           

In [22]:
sub_products_to_consider = reduced_subissues['Sub-product'].value_counts()[reduced_subissues['Sub-product'].value_counts() > 100].index
final_df_2023 = reduced_subissues[reduced_subissues['Sub-product'].isin(sub_products_to_consider)]

In [23]:
final_df_2023.shape

(247517, 5)

### Value counts

In [24]:
final_df_2023['Product'].value_counts()

Product
Credit Reporting               211695
Checking or savings account     12285
Credit/Prepaid Card             11975
Debt collection                  9380
Loans / Mortgage                 2182
Name: count, dtype: int64

In [25]:
final_df_2023['Sub-product'].value_counts()

Sub-product
Credit reporting                              210735
General-purpose credit card or charge card     10668
Checking account                               10409
Other debt                                      3041
I do not know                                   2316
Credit card debt                                1652
Federal student loan servicing                  1344
Store credit card                               1307
Medical debt                                    1053
Savings account                                  989
Other personal consumer report                   960
Loan                                             732
Other banking product or service                 725
Auto debt                                        581
Telecommunications debt                          419
Rental debt                                      179
CD (Certificate of Deposit)                      162
Mortgage debt                                    139
Conventional home mortgage        

In [27]:
final_df_2023['Issue'].value_counts()

Issue
Incorrect information on your report                                                87200
Improper use of your report                                                         61868
Problem with a credit reporting company's investigation into an existing problem    45371
Problem with a company's investigation into an existing problem                     20985
Managing an account                                                                  7367
Attempts to collect debt not owed                                                    5453
Problem with a purchase shown on your statement                                      3253
Written notification about debt                                                      2404
Closing an account                                                                   1975
Problem with a lender or other company charging your account                         1378
Dealing with your lender or servicer                                                 1293
Unab

In [28]:
final_df_2023['Sub-issue'].value_counts()

Sub-issue
Information belongs to someone else                                                      57850
Reporting company used your report improperly                                            48732
Their investigation did not fix an error on your report                                  45395
Credit inquiries on your report that you don't recognize                                 13136
Account status incorrect                                                                 10208
Account information incorrect                                                             9267
Was not notified of investigation status or results                                       9200
Investigation took more than 30 days                                                      8928
Personal information incorrect                                                            5900
Debt is not yours                                                                         2785
Deposits and withdrawals                

### Unique categories

In [29]:
print(f"Unique Product offerings: {final_df_2023['Product'].nunique()}")
print(f"Unique Sub-product offerings: {final_df_2023['Sub-product'].nunique()}")
print(f"Unique Issue offerings: {final_df_2023['Issue'].nunique()}")
print(f"Unique Sub-issue offerings: {final_df_2023['Sub-issue'].nunique()}")

Unique Product offerings: 5
Unique Sub-product offerings: 19
Unique Issue offerings: 25
Unique Sub-issue offerings: 44


### Preparing the train and test splits

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
final_df_2023.head()

,Consumer complaint narrative,Product,Sub-product,Issue,Sub-issue
1,I have previously disputed this item with you ...,Credit Reporting,Credit reporting,Problem with a company's investigation into an...,Investigation took more than 30 days
2,I kindly request that you update my credit rep...,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft
3,I implore you to conduct a comprehensive inves...,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft
4,In accordance with the Fair Credit Reporting A...,Credit Reporting,Credit reporting,Incorrect information on your report,Information belongs to someone else
5,In accordance with Fair c=Credit Reporting Act...,Credit Reporting,Credit reporting,Improper use of your report,Reporting company used your report improperly


In [86]:
X = final_df_2023['Consumer complaint narrative']
y = final_df_2023[['Product','Sub-product','Issue','Sub-issue']]

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y['Product'],test_size=0.25,random_state=42)

In [91]:
train_df = pd.concat([X_train,y_train],axis = 1).reset_index(drop = True)
test_df = pd.concat([X_test,y_test],axis = 1).reset_index(drop = True)

In [92]:
train_df.head()

,Consumer complaint narrative,Product,Sub-product,Issue,Sub-issue
0,The credit bureaus keep disrespecting the laws...,Credit Reporting,Credit reporting,Problem with a company's investigation into an...,Their investigation did not fix an error on yo...
1,I sent in a complaint in XXXX of 2021 about so...,Credit Reporting,Credit reporting,Incorrect information on your report,Information belongs to someone else
2,I ordered a copy of my report and I found out ...,Credit Reporting,Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...
3,It appears that my credit file has been compro...,Credit Reporting,Credit reporting,Incorrect information on your report,Information belongs to someone else
4,"I have never authorized, consented to nor bene...",Credit Reporting,Credit reporting,Incorrect information on your report,Information belongs to someone else


In [94]:
train_df.shape

(185637, 5)

In [95]:
test_df.shape

(61880, 5)

In [99]:
import os

directory_to_save = './data_splits/'

if not os.path.exists(directory_to_save):
    os.makedirs(directory_to_save)

train_df.to_csv(directory_to_save + 'train-data-split.csv',index = False)
test_df.to_csv(directory_to_save + 'test-data-split.csv',index = False)